In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vehicle-sales-data/car_prices.csv
/kaggle/input/electric-vehicle-population-size-2024/Electric_Vehicle_Population_Size_History_By_County_.csv
/kaggle/input/lamborghini-sales-and-stocks/LamboData.xlsx
/kaggle/input/lamborghini-sales-and-stocks/Lambo-Future.csv


In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
# Read the data
df = pd.read_csv('../input/vehicle-sales-data/car_prices.csv')
    

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558837 entries, 0 to 558836
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          558837 non-null  int64  
 1   make          548536 non-null  object 
 2   model         548438 non-null  object 
 3   trim          548186 non-null  object 
 4   body          545642 non-null  object 
 5   transmission  493485 non-null  object 
 6   vin           558833 non-null  object 
 7   state         558837 non-null  object 
 8   condition     547017 non-null  float64
 9   odometer      558743 non-null  float64
 10  color         558088 non-null  object 
 11  interior      558088 non-null  object 
 12  seller        558837 non-null  object 
 13  mmr           558799 non-null  float64
 14  sellingprice  558825 non-null  float64
 15  saledate      558825 non-null  object 
dtypes: float64(4), int64(1), object(11)
memory usage: 68.2+ MB


**Data Overview:**

Size: A total of 558837 Rows of data
Features: 13 Total Features

Quantitative Features (Numerical):
*     year(int64): Vehicle Manufacturing Year.
*     sellingprice(float64): Sales price of the viehicle in question.
*     condition(float64): Condition of the vehicle (probably integers).
*     odometer(float64): Miles recorded on odometer at selling time.
*     mmr(float64): Manheim Market Report Value (estimated market value)

Qualitative Features:
*     make (object): The vehicle's brand/manufacturer.
*     model (object): The vehicle's specific model.
*     trim (object): Additional specification within the model.
*     body (object): The vehicle's body type (e.g., Sedan, SUV, etc.).
*     transmission (object): The type of transmission (e.g., automatic, manual).
*     vin (object): The vehicle's unique Vehicle Identification Number.
*     state (object): The state where the vehicle is registered.
*     color (object): The vehicle's exterior color.
*     interior (object): The vehicle's interior color.
*     seller (object): The entity that sold the vehicle.
*     saledate (object): The date and time of the sale.

In [33]:
print(data.isnull().sum())

year                0
make            10301
model           10399
trim            10651
body            13195
transmission    65352
vin                 4
state               0
condition       11820
odometer           94
color             749
interior          749
seller              0
mmr                38
sellingprice       12
saledate           12
dtype: int64


**Missing Data Solution**
Given that there's such a large number of columns with missing data it would not be suitable to drop the entire columns of data that have missing values. Limitations on RAM and CPU keep me from using a full tranformation and imputation technique since the data set is so large. Instead I can use data.loc to make the transformations. I will be using the mode for all of the catgorical transformations and the mean for any numerical tansformations.

In [34]:
import pandas as pd
import warnings

# Suppress the warning (Warning given after first run through)
warnings.filterwarnings("ignore", category=FutureWarning)


# Impute missing values
df.loc[:, 'make'].fillna(df['make'].mode()[0], inplace=True)
df.loc[:, 'model'].fillna(df['model'].mode()[0], inplace=True)
df.loc[:, 'body'].fillna(df['body'].mode()[0], inplace=True)
df.loc[:, 'trim'].fillna(df['trim'].mode()[0], inplace=True)
df.loc[:, 'color'].fillna(df['color'].mode()[0], inplace=True)
df.loc[:, 'interior'].fillna(df['interior'].mode()[0], inplace=True)
df.loc[:, 'transmission'].fillna(df['transmission'].mode()[0], inplace=True)
df.loc[:, 'condition'].fillna(df['condition'].mean(), inplace=True)
df.loc[:, 'odometer'].fillna(df['odometer'].mean(), inplace=True)
df.loc[:, 'mmr'].fillna(df['mmr'].mean(), inplace=True)

# Drop rows with missing 'sellingprice'
df.dropna(subset=['sellingprice'], axis='rows', inplace=True)

# Split data into features (X) and target variable (y)
X = df.drop('sellingprice', axis=1)
y = df['sellingprice']

# Re-enable the warning
warnings.filterwarnings("default", category=FutureWarning)


Because of errors later in the process I had to use a different method because not all of the vin rows were deleted and the make column was not correctly transformed.

In [55]:
# Fill missing values in categorical columns with mode
categorical_cols = ['make', 'model', 'body', 'trim', 'color', 'interior', 'transmission']
for col in categorical_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

# Fill missing values in numerical columns with mean
numerical_cols = ['condition', 'odometer', 'mmr']
for col in numerical_cols:
    df[col].fillna(df[col].mean(), inplace=True)

# Drop rows with missing 'sellingprice' values
df.dropna(subset=['sellingprice'], axis=0, inplace=True)

/tmp/ipykernel_64/3304489401.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)
/tmp/ipykernel_64/3304489401.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

In [56]:

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# Shape of training data (num_rows, num_columns)
print(X_train_full.shape)

print(X_train_full.isnull().sum())



(447060, 15)
year            0
make            0
model           0
trim            0
body            0
transmission    0
vin             4
state           0
condition       0
odometer        0
color           0
interior        0
seller          0
mmr             0
saledate        0
dtype: int64


**Missing values almost fixed with the exception of vin**


In [59]:
print(X_train_full.shape)

# Check for missing values in training data
print(X_train_full.isnull().sum())

# Drop rows with missing 'vin' values
X_train_full.dropna(subset=['vin'], inplace=True)

# Check the shape of the data after dropping rows
print(X_train_full.shape)

(447056, 15)
year            0
make            0
model           0
trim            0
body            0
transmission    0
vin             0
state           0
condition       0
odometer        0
color           0
interior        0
seller          0
mmr             0
saledate        0
dtype: int64
(447056, 15)


In [58]:
# Define model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
model = RandomForestRegressor(n_estimators=100, random_state=0)

model.fit(X_train_full, y_train)

ValueError: could not convert string to float: 'Hyundai'

In [ ]:
print(X_train_full.shape)

# Check for missing values in training data
print(X_train_full.isnull().sum())

# Drop rows with missing 'vin' values
X_train_full.dropna(subset=['vin'], inplace=True)

# Check the shape of the data after dropping rows
print(X_train_full.shape)

In [60]:
print(df['make'].dtype)

# Check unique values in 'make' column
print(df['make'].unique())



object
['Kia' 'BMW' 'Volvo' 'Nissan' 'Chevrolet' 'Audi' 'Ford' 'Hyundai' 'Buick'
 'Cadillac' 'Acura' 'Lexus' 'Infiniti' 'Jeep' 'Mercedes-Benz' 'Mitsubishi'
 'Mazda' 'MINI' 'Land Rover' 'Lincoln' 'lincoln' 'Jaguar' 'Volkswagen'
 'Toyota' 'Subaru' 'Scion' 'Porsche' 'bmw' 'Dodge' 'FIAT' 'Chrysler'
 'ford' 'Ferrari' 'Honda' 'GMC' 'mitsubishi' 'Ram' 'smart' 'chevrolet'
 'Bentley' 'chrysler' 'pontiac' 'Pontiac' 'Saturn' 'Maserati' 'Mercury'
 'HUMMER' 'landrover' 'cadillac' 'land rover' 'mercedes' 'mazda' 'toyota'
 'lexus' 'gmc truck' 'honda' 'nissan' 'porsche' 'Saab' 'Suzuki' 'dodge'
 'subaru' 'Oldsmobile' 'oldsmobile' 'hyundai' 'jeep' 'Isuzu' 'dodge tk'
 'Geo' 'acura' 'volkswagen' 'suzuki' 'kia' 'audi' 'Rolls-Royce' 'gmc'
 'maserati' 'mazda tk' 'mercury' 'buick' 'hyundai tk' 'mercedes-b' 'vw'
 'Daewoo' 'chev truck' 'ford tk' 'plymouth' 'Plymouth' 'ford truck'
 'Tesla' 'airstream' 'dot' 'Aston Martin' 'Fisker' 'Lamborghini' 'Lotus']


Seems like bmw repeats as a unique value with BMW as a capital, need to make them all capital.

In [62]:
df['make'] = df['make'].str.capitalize()

print(df['make'].unique())

['Kia' 'Bmw' 'Volvo' 'Nissan' 'Chevrolet' 'Audi' 'Ford' 'Hyundai' 'Buick'
 'Cadillac' 'Acura' 'Lexus' 'Infiniti' 'Jeep' 'Mercedes-benz' 'Mitsubishi'
 'Mazda' 'Mini' 'Land rover' 'Lincoln' 'Jaguar' 'Volkswagen' 'Toyota'
 'Subaru' 'Scion' 'Porsche' 'Dodge' 'Fiat' 'Chrysler' 'Ferrari' 'Honda'
 'Gmc' 'Ram' 'Smart' 'Bentley' 'Pontiac' 'Saturn' 'Maserati' 'Mercury'
 'Hummer' 'Landrover' 'Mercedes' 'Gmc truck' 'Saab' 'Suzuki' 'Oldsmobile'
 'Isuzu' 'Dodge tk' 'Geo' 'Rolls-royce' 'Mazda tk' 'Hyundai tk'
 'Mercedes-b' 'Vw' 'Daewoo' 'Chev truck' 'Ford tk' 'Plymouth' 'Ford truck'
 'Tesla' 'Airstream' 'Dot' 'Aston martin' 'Fisker' 'Lamborghini' 'Lotus']


In [64]:
# Filling in missing with mode again to help resolve

mode_make = df['make'].mode()[0]
df['make'].fillna(mode_make, inplace=True)

print(df['make'].isnull().sum())

0


/tmp/ipykernel_64/672323764.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['make'].fillna(mode_make, inplace=True)


Looks good now.

**Next Steps**

Above I programmed a pipeline to transform the numerical and categorical columns. Next I plan on using machine learning to make predictions on the sales prices of vehicles and use the Mean Average error as an indicator of prediction performance. This is the first time I've done this on a new data set so I assume the MAE will be high.

In [65]:
# Define model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
model = RandomForestRegressor(n_estimators=100, random_state=0)



Ran into an error on this step because python needs my data to be in a data frame instead of a numpy array. Went back and fixed issues. This time there were no errors. Now I'm going to try to fit my model on the X_train_full

In [66]:
model.fit(X_train_full, y_train)

ValueError: could not convert string to float: 'Hyundai'

In [ ]:
It seems like even after I transformed the data based on the mode I still need one hot encoding to run the random forest.

In [67]:
categorical_columns = df.select_dtypes(include=['object']).columns
print(categorical_columns)

Index(['make', 'model', 'trim', 'body', 'transmission', 'vin', 'state',
       'color', 'interior', 'seller', 'saledate'],
      dtype='object')


In [ ]:
dummy_df = pd.get_dummies(df, columns=['make', 'model', 'trim', 'body', 'transmission', 'vin', 'state', 'color', 'interior', 'seller', 'saledate'])


In [68]:
# Now for another split, this time using the data frame.

# Split data into features (X) and target variable (y)
X = dummy_df.drop('sellingprice', axis=1)
y = dummy_df['sellingprice']

# Split data into training and validation sets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

NameError: name 'dummy_df' is not defined

In [51]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Define categorical columns
categorical_cols = ['make', 'model', 'trim', 'body', 'transmission', 'state', 'color', 'interior', 'seller']

# Preprocessing for categorical data
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_cols)
    ])


/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0, 1, 2, 3, 5, 6, 8, 9] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [ ]:
X_train_preprocessed = preprocessor.fit_transform(X_train_full)
X_valid_preprocessed = preprocessor.transform(X_valid_full)

# Train the model
model.fit(X_train_preprocessed, y_train)

# Predictions on the validation set
predictions = model.predict(X_valid_preprocessed)

# Calculate Mean Absolute Error
mae = mean_absolute_error(y_valid, predictions)
print('Mean Absolute Error:', mae)

Too much CPU and RAM Used. Even tried running the model on an instance. Still not working. Shutting this project down to work on a smaller data set.